In [2]:
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [3]:
train_2016 = pd.read_csv('https://drive.google.com/uc?id=15GlcdLJ79bc5_WhVNViepQaXvsE1vYb8')
properties_2016 = pd.read_csv('C:/Users/Juhi Singhal/Downloads/properties_2016.csv')

C:\Users\Juhi Singhal\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
training_data = pd.merge(train_2016, properties_2016, on=['parcelid'], how='inner')
training_data.shape

(90275, 60)

In [5]:
# Lets create a transformer
## transformer, dataframe for column, outcome waht we want from eda
## actually take model, buid the model and transform it. 
## save a model and tae model, build a server. 
class BinaryNullTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.assign(**{col: X[col].notnull() for col in self.columns})

In [6]:
# Let's try it and compare with original data
exist_or_not_variables = ['garagecarcnt', 'yardbuildingsqft26', 'basementsqft', 'fireplacecnt', 'yardbuildingsqft17']
transformer = BinaryNullTransformer(exist_or_not_variables)
transformed_data = transformer.fit_transform(training_data.head(20))
transformed_data[exist_or_not_variables].rename(
    columns={col: f'{col}_transformed' for col in exist_or_not_variables}
).join(training_data.head(20)[exist_or_not_variables])


,garagecarcnt_transformed,yardbuildingsqft26_transformed,basementsqft_transformed,fireplacecnt_transformed,yardbuildingsqft17_transformed,garagecarcnt,yardbuildingsqft26,basementsqft,fireplacecnt,yardbuildingsqft17
0,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
1,True,False,False,False,False,2.0,NaN,NaN,NaN,NaN
2,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
3,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
4,True,False,False,False,False,2.0,NaN,NaN,NaN,NaN
5,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
6,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN
7,True,False,False,True,False,2.0,NaN,NaN,1.0,NaN
8,True,False,False,False,False,1.0,NaN,NaN,NaN,NaN
9,True,False,False,False,False,1.0,NaN,NaN,NaN,NaN


In [8]:
# Let's create another transformer
class IntervalCategorizer(BaseEstimator, TransformerMixin):
    def __init__(self, column, rng=(2,4)):
        self.column = column
        self.rng = rng
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.assign(**{self.column: np.where((X[self.column] >= self.rng[0]) & (X[self.column] <= self.rng[1]), True, False)})

In [10]:
# Let's try it again and compare with original data
transformer = IntervalCategorizer('bedroomcnt')
transformed_data = transformer.fit_transform(training_data.head(20))
transformed_data[['bedroomcnt']].rename(columns={'bedroomcnt': 'transformed'}).join(training_data.head(20)[['bedroomcnt']])

,transformed,bedroomcnt
0,True,3.0
1,True,4.0
2,True,2.0
3,True,2.0
4,True,4.0
5,True,4.0
6,True,2.0
7,True,3.0
8,True,2.0
9,True,2.0


In [11]:
# Last transformer
class Normalizer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        self.means = X[self.columns].mean()
        self.std = X[self.columns].std()
        
        return self
    
    def transform(self, X):
        return X.assign(**{col: (X[col] - self.means[col]) / self.std[col] for col in self.columns})

In [12]:
# Let's try it one last time and compare with original data
transformer = Normalizer(['finishedsquarefeet12', 'structuretaxvaluedollarcnt'])
transformed_data = transformer.fit_transform(training_data.head(20))
transformed_data[['bedroomcnt']].rename(columns={'bedroomcnt': 'transformed'}).join(training_data.head(20)[['bedroomcnt']])

,transformed,bedroomcnt
0,3.0,3.0
1,4.0,4.0
2,2.0,2.0
3,2.0,2.0
4,4.0,4.0
5,4.0,4.0
6,2.0,2.0
7,3.0,3.0
8,2.0,2.0
9,2.0,2.0


In [ ]:
# Transformer to select our variables
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

In [ ]:
# issue - we now have 4 normalizers, and it's a pain to actually run these manually everytime
# solution - pipelines.  Pipelines take a set of transforms and a final estimator.  Let's see how this works
exist_or_not_variables = ['garagecarcnt', 'yardbuildingsqft26', 'basementsqft', 'fireplacecnt', 'yardbuildingsqft17']
bedrooms = ['bedroomcnt']
bathrooms = ['fullbathcnt']
normalized_variables = ['finishedsquarefeet12', 'structuretaxvaluedollarcnt']
other_variables = ['yearbuilt']

transformer_pipeline = Pipeline(
    steps=[
        ('binary_null', BinaryNullTransformer(exist_or_not_variables)),
        ('bedrooms', IntervalCategorizer(*bedrooms, (2, 4))),
        ('bathrooms', IntervalCategorizer(*bathrooms, (2, 4))),
        ('normalize', Normalizer(normalized_variables)),
        ('select_features', FeatureSelector([*exist_or_not_variables, *bedrooms, *bathrooms, *normalized_variables, *other_variables]))
    ]
)

In [ ]:
# let's try using the pipeline
transformed_data = transformer_pipeline.fit_transform(training_data.head(20))

# transformed_data[exist_or_not_variables].rename(
#     columns={col: f'{col}_transformed' for col in exist_or_not_variables}
# ).join(training_data.head(20)[exist_or_not_variables])

transformed_data